# Have Several Agents Collaborate in a Multi-Agent Hierarchy

In this example, we will make a **multi-agent web browser** - an agentic system with several agents collaborating to solve problems using the web.

The system has a simple hierarchy, using a `ManagedAgent` object to wrap the managed web search agent:
```
              +----------------+
              | Manager agent  |
              +----------------+
                       |
        _______________|______________
       |                              |
  Code interpreter   +--------------------------------+
       tool          |         Managed agent          |
                     |      +------------------+      |
                     |      | Web Search agent |      |
                     |      +------------------+      |
                     |         |            |         |
                     |  Web Search tool     |         |
                     |             Visit webpage tool |
                     +--------------------------------+
```

## Setups

In [ ]:
!pip install markdownify duckduckgo-search smolagents

## Select AI models

Our agent will be powered by [`Qwen/Qwen2.5-72B-Instruct`](https://huggingface.co/Qwen/Qwen2.5-72B-Instruct) using `HfApiEngine` class that uses HF's Inference API.

In [ ]:
model_id = 'Qwen/Qwen2.5-72B-Instruct'

## Create a web search tool

For web browsing, we can use the `DuckDuckGoSearchTool` tool to provide a Google search eequivalent. We will also need the `VisitWebpageTool` to peak into the page found by `DuckDuckGoSearchTool`.

In [ ]:
import re
import requests
from markdownify import markdownify as md
from requests.exceptions import RequestException
from smolagents import tool


@tool
def visit_webpage(url: str) -> str:
    """Visit a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the url
        response = requests.get(url)
        # Raise an exception for bad status codes
        response.raise_for_status()

        # Convert the HTML content to Markdown
        markdown_content = md(response.text).strip()
        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

In [ ]:
# Test the tool
print(visit_webpage("https://en.wikipedia.org/wiki/Hugging_Face")[:500])

## Build our multi-agent system

Web browsing is a single-timeline task that does not require parallel tool calls, so JSON tool calling works well for this. Thus, we choose a `ReactJsonAgent`.

Also, since sometimes web search requires exploring many pages before finding the correct answer, we prefer to increase the number of `max_iterations` to 10.

In [ ]:
from smolagents import CodeAgent, ToolCallingAgent, HfApiModel, ManagedAgent, DuckDuckGoSearchTool

model = HfApiModel(model_id)

In [ ]:
web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), visit_webpage],
    model=model,
    max_iterations=10,
)

Next we will wrap this agent into a `ManagedAgent` that will make it callable by its manager agent.

In [ ]:
managed_web_agent = ManagedAgent(
    agent=web_agent,
    name='search_agent',
    description='Runs web searches for you. Give it your query as an argument.'
)

Finally we will create a manager agent, and then we will pass our managed agent to it in its `managed_agents` argument.

Since this agent is the one tasked with the planning and thinking, advanced reasoning will be beneficial, so a `ReactCodeAgent` will be the best choice. Also, we want to ask a question that involves the current year, so we will add `additional_authorized_imports=['time', 'datetime']`.

In [ ]:
manager_agnet = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[managed_web_agent],
    additional_authorized_imports=['time', 'datetime']
)

In [ ]:
# Test our system
query = "How many years ago was Stripe founded?"
manager_agent.run(query)